In [1]:
from espn_api.basketball import League, Matchup, Team

In [2]:
espn_s2 = "AECc%2BKMx5L8l9Rai5PHG%2FT%2Fjtw%2BR%2FZozX391uxpta1J6tD61%2F4NV88%2BVPlCSe0Wjcg0J9jJ0oTJ%2BMkehJLT8MSDfgiubWB9F1n3H%2BnTHWc9OTkzM2ceQFGPiA%2B9AF4yQzt0I3%2FdCgk%2BdQLbgybhS6y1yRS9NIDyfXMSgcqFqRbn5F7zBTzwu9A%2BD2Pi6g6KMR29PbmNbtxSrCJkfbF%2BaZtrpqDbHBZhLyfqOAhQIDx0q1iD0u1DkhxVNoRnxx%2FmEvS%2FHEJZWmfvZM%2F0IAHxPrfVx"
swid="{3C4A75B6-F84B-48AE-8A75-B6F84B48AE47}"

In [3]:
league = League(league_id=30695, year=2021, espn_s2=espn_s2, swid=swid)

DAY_TO_DAY
DAY_TO_DAY
DAY_TO_DAY
OUT
DAY_TO_DAY
DAY_TO_DAY
DAY_TO_DAY
DAY_TO_DAY
OUT
DAY_TO_DAY
OUT
OUT
DAY_TO_DAY
DAY_TO_DAY
OUT
DAY_TO_DAY
DAY_TO_DAY
OUT
OUT
OUT
DAY_TO_DAY


In [4]:
MY_TEAM_ID = 2
schedule = league.get_team_data(MY_TEAM_ID).schedule

In [5]:
league.recent_activity()

[Activity((Team(Tim Liang),FA ADDED,Paul Millsap) (Team(Tim Liang),DROPPED,Malik Beasley)),
 Activity((Team(PHI Lower Merion 33),FA ADDED,Naz Reid) (Team(PHI Lower Merion 33),DROPPED,Jae Crowder)),
 Activity((Team(NY Zhanos Diesel),DROPPED,Hassan Whiteside)),
 Activity((Team(Team Tao),FA ADDED,De'Andre Hunter)),
 Activity((Team(PHI Lower Merion 33),FA ADDED,Kyle Anderson) (Team(PHI Lower Merion 33),DROPPED,Shake Milton)),
 Activity((Team(Tim Liang),DROPPED,Derrick Rose) (Team(Tim Liang),FA ADDED,DeAndre Jordan)),
 Activity((Team(Shanghai SHARKS),DROPPED,Seth Curry) (Team(Shanghai SHARKS),FA ADDED,Luguentz Dort)),
 Activity((Team(Team Beyond),DROPPED,Kevin Love) (Team(Team Beyond),FA ADDED,Dillon Brooks)),
 Activity((Team(Shanghai SHARKS),WAIVER ADDED,Nicolas Batum) (Team(Shanghai SHARKS),DROPPED,Isaac Okoro)),
 Activity((Team(Badwomen ``),DROPPED,Joe Harris) (Team(Badwomen ``),WAIVER ADDED,Andrew Wiggins)),
 Activity((Team(SEA MoNeYPro),FA ADDED,Darius Bazley) (Team(SEA MoNeYPro),DROPP

In [6]:
from activity.myUtil import get_opponent_team, get_my_team

In [7]:
opponent_roster = get_opponent_team(schedule[1], MY_TEAM_ID).roster

In [8]:
(opponent_roster[-2]).proTeam

'BKN'

In [9]:
from espn_api.basketball.week import Week
from espn_api.basketball.roster_week_predictor import RosterWeekPredictor

In [10]:
currWeek = Week(league, 2)

In [11]:
predicted_points_team_name_map = {}
number_of_games_team_name_map = {}
for team in league.teams:
    predictor = RosterWeekPredictor(team.roster, currWeek)
    predicted_points_team_name_map[team.team_name] = predictor.predict()
    number_of_games_team_name_map[team.team_name] = predictor.get_total_number_of_games()

In [12]:
def get_tuple_average(tup):
    return (tup[0] + tup[1]) / 2

In [13]:
team_scores = {}
for team in league.teams:
    predicted_points = predicted_points_team_name_map[team.team_name]
    team_scores[team.team_name] = predicted_points[0], predicted_points[1], get_tuple_average(predicted_points)

In [14]:
table_output = []
for team_name, scores in team_scores.items():
    lo, hi, avg = scores
    table_output.append((team_name, number_of_games_team_name_map[team_name], lo, hi, avg))
table_output.sort(reverse=True, key=lambda x: x[-1])

In [15]:
from IPython.display import HTML, display
import tabulate
table_output.insert(0, ("Team Name", "# of games", "Low", "High", "Avg"))
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Team Name,# of games,Low,High,Avg
SEA MoNeYPro,40,1167.859164521359,1585.6127840909091,1376.735974306134
CZ ButtKicker,39,1043.8703943133864,1589.61124324877,1316.7408187810784
Badwomen ``,38,1111.3649667931688,1453.8705893770389,1282.617778085104
Tim Liang,39,986.9339795788381,1452.4959392708683,1219.7149594248533
Dragon City Tiedan,38,1044.8617239816444,1351.55001667794,1198.205870329792
Team Tao,37,993.8530857616679,1364.5230238295462,1179.188054795607
Team Beyond,39,1010.0213813607281,1321.4089290584186,1165.7151552095734
PHI Lower Merion 33,31,887.5007137916673,1243.0014652014652,1065.2510894965662
Shanghai SHARKS,32,787.9530948495377,1171.4950320512821,979.7240634504099
NY Zhanos Diesel,30,879.3939959189418,1065.2972276094033,972.3456117641725


In [16]:
match_up_points = []
match_up_points.append(["Home Team", "Estimate Points", "# of Games", "Away Team", "Estimate Points", "# of Games", "+/-"])
for matchup in league.scoreboard():
    home_team_average = team_scores[matchup.home_team.team_name][-1]
    away_team_average = team_scores[matchup.away_team.team_name][-1]
    match_up_points.append([matchup.home_team.team_name, home_team_average, number_of_games_team_name_map[matchup.home_team.team_name],
                            matchup.away_team.team_name, away_team_average, number_of_games_team_name_map[matchup.away_team.team_name],
                           home_team_average - away_team_average])

In [17]:
display(HTML(tabulate.tabulate(match_up_points, tablefmt='html')))

Home Team,Estimate Points,# of Games,Away Team,Estimate Points,# of Games,+/-
SEA MoNeYPro,1376.735974306134,40,Dragon City Tiedan,1198.205870329792,38,178.53010397634193
Team Tao,1179.188054795607,37,PHI Lower Merion 33,1065.2510894965662,31,113.93696529904082
Shanghai SHARKS,979.7240634504099,32,CZ ButtKicker,1316.7408187810784,39,-337.0167553306685
Team Beyond,1165.7151552095734,39,Tim Liang,1219.7149594248533,39,-53.99980421527994
Badwomen ``,1282.617778085104,38,NY Zhanos Diesel,972.3456117641725,30,310.27216632093143


In [18]:
opponent_roster[0].injuryStatus

'ACTIVE'

In [19]:
league._get_pro_schedule(1)

{12: (13, 1608692400000),
 13: (12, 1608692400000),
 17: (9, 1608681600000),
 9: (17, 1608681600000)}

In [24]:
currWeek.team_game_list

[['LAL', 'BKN', 'POR', 'OKC', 'UTA', 'MEM', 'ATL', 'DEN', 'DET', 'HOU'],
 ['IND',
  'LAC',
  'MIA',
  'MIL',
  'MIN',
  'NYK',
  'ORL',
  'PHL',
  'PHO',
  'SAC',
  'OKC',
  'WAS',
  'TOR',
  'BOS',
  'NOP',
  'CHI',
  'CLE',
  'DEN',
  'DET',
  'GSW'],
 ['LAC',
  'LAL',
  'MIA',
  'MIL',
  'BKN',
  'POR',
  'SAS',
  'MEM',
  'CHA',
  'ATL',
  'BOS',
  'DAL'],
 ['IND',
  'NYK',
  'ORL',
  'PHL',
  'PHO',
  'SAC',
  'OKC',
  'UTA',
  'WAS',
  'TOR',
  'NOP',
  'CHI',
  'CLE',
  'HOU'],
 ['LAC',
  'LAL',
  'MIA',
  'MIL',
  'MIN',
  'BKN',
  'PHO',
  'POR',
  'SAS',
  'UTA',
  'WAS',
  'MEM',
  'CHA',
  'ATL',
  'BOS',
  'CHI',
  'DAL',
  'DEN',
  'DET',
  'GSW'],
 ['IND',
  'NYK',
  'ORL',
  'PHL',
  'SAC',
  'OKC',
  'TOR',
  'CHA',
  'ATL',
  'NOP',
  'CLE',
  'HOU'],
 ['LAC',
  'LAL',
  'MIN',
  'BKN',
  'PHO',
  'POR',
  'SAS',
  'UTA',
  'WAS',
  'MEM',
  'BOS',
  'CHI',
  'DAL',
  'DEN',
  'DET',
  'GSW']]

In [27]:
currWeek=Week(league, 1)
currWeek.scoring_period

(1, 6)

In [23]:
from espn_api.basketball.roster_week_predictor import RosterWeekPredictor

In [24]:
predictor2 = RosterWeekPredictor(my_roster, currWeek)

In [25]:
predictor2.predict()

(1677.7853361518614, 0)

In [23]:
a = [1,2,3]
a.sort(reverse=True)
a

[3, 2, 1]